In [74]:
%run header.py
%matplotlib inline

from utils import sample_from_interactions
from utils import from_interactions_to_coo
from utils import combine_with_identity

In [2]:
exp_path = '/nmnt/x04-hdd/boris_temp/SGIMC_IMC/movielens/'

In [3]:
PROBLEM = "classification" if False else "regression"

step_fn = step_qaadmm

if PROBLEM == "classification":
    QAObjectiveLoss = QAObjectiveLogLoss
else:
    QAObjectiveLoss = QAObjectiveL2Loss  # QAObjectiveHuberLoss

## Data loading

In [5]:
interactions = np.load(exp_path + 'I.npy')
X = np.load(exp_path + 'X.npy')
Y = np.load(exp_path + 'Y.npy')

X_comb = combine_with_identity(X)
Y_comb = combine_with_identity(Y)

In [6]:
R_full = from_interactions_to_coo(interactions)
full_mask = R_full.toarray() > 0

## n_elem = 0.8

In [130]:
C_ridge = 1e-4
C_group = 3e-4
C_lasso = 10 * C_group
C = (C_lasso, C_group, C_ridge)

eta = 5e0

step_kwargs = {
    "C": C,                 # the regularizr constants (C_lasso, C_group, C_ridge)
    "eta": eta,             # the eta of the ADMM (larger - faster but more unstable)
    "rtol": 1e-4,           # the relative tolerance for stopping the ADMM
    "atol": 1e-7,           # the absolute tolerance
    "method": "cg",         # the method to use in Sub_0
    "n_iterations": 2,      # the number of iterations of the inner ADMM
}

n_iter = 100
K = 20

n_elem = 0.8

In [87]:
interaction_train, interaction_test = sample_from_interactions(interactions, n_elem)

R_train = from_interactions_to_coo(interaction_train, R_shape=R_full.shape).tocsr()
train_mask = R_train.toarray() > 0
test_mask = from_interactions_to_coo(interaction_test, R_shape=R_full.shape).toarray() > 0

In [131]:
problem = IMCProblem(QAObjectiveLoss, X_comb, Y_comb, R_train, n_threads=8)

W_0 = random_state.normal(size=(X_comb.shape[1], K))
H_0 = random_state.normal(size=(Y_comb.shape[1], K))

W, H = W_0.copy(), H_0.copy()

W, H = imc_descent(problem, W, H,
                   step_fn,                  # the inner optimization
                   step_kwargs=step_kwargs,  # asrtguments for the inner optimizer
                   n_iterations=n_iter,      # the number of outer iterations (Gauss-Siedel)
                   n_init_iterations=0,
                   return_history=True,      # Record the evolution of the matrices (W, H)
                   rtol=1e-5,                # relative stopping tolerance for the outer iterations
                   atol=1e-7,                # absolute tolerance
                   verbose=True,             # show the progress bar
                   check_product=True,       # use the product W H' for stopping
                   )

0it [00:00, ?it/s]
100%|██████████| 100/100 [00:15<00:00,  6.74it/s]


In [132]:
R_hat = get_prediction(X_comb, W, H, Y_comb)

print(relative_loss(R_full.toarray(), R_hat, mask=test_mask))
print(relative_loss(R_full.toarray(), R_hat, mask=train_mask))

0.1097722307428194
0.07387231049226703


## n_elem = 0.7

In [180]:
C_ridge = 1e-4
C_group = 3e-4
C_lasso = 10 * C_group
C = (C_lasso, C_group, C_ridge)

eta = 5e0

step_kwargs = {
    "C": C,                 # the regularizr constants (C_lasso, C_group, C_ridge)
    "eta": eta,             # the eta of the ADMM (larger - faster but more unstable)
    "rtol": 1e-4,           # the relative tolerance for stopping the ADMM
    "atol": 1e-7,           # the absolute tolerance
    "method": "cg",         # the method to use in Sub_0
    "n_iterations": 2,      # the number of iterations of the inner ADMM
}

n_iter = 100
K = 10

n_elem = 0.7

In [181]:
interaction_train, interaction_test = sample_from_interactions(interactions, n_elem)

R_train = from_interactions_to_coo(interaction_train, R_shape=R_full.shape).tocsr()
train_mask = R_train.toarray() > 0
test_mask = from_interactions_to_coo(interaction_test, R_shape=R_full.shape).toarray() > 0

/nmnt/media/home/boris/env3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [182]:
problem = IMCProblem(QAObjectiveLoss, X_comb, Y_comb, R_train, n_threads=8)

W_0 = random_state.normal(size=(X_comb.shape[1], K))
H_0 = random_state.normal(size=(Y_comb.shape[1], K))

W, H = W_0.copy(), H_0.copy()

W, H = imc_descent(problem, W, H,
                   step_fn,                  # the inner optimization
                   step_kwargs=step_kwargs,  # asrtguments for the inner optimizer
                   n_iterations=n_iter,      # the number of outer iterations (Gauss-Siedel)
                   n_init_iterations=0,
                   return_history=True,      # Record the evolution of the matrices (W, H)
                   rtol=1e-5,                # relative stopping tolerance for the outer iterations
                   atol=1e-7,                # absolute tolerance
                   verbose=True,             # show the progress bar
                   check_product=True,       # use the product W H' for stopping
                   )

0it [00:00, ?it/s]
100%|██████████| 100/100 [00:08<00:00, 11.79it/s]


In [183]:
R_hat = get_prediction(X_comb, W, H, Y_comb)

print(relative_loss(R_full.toarray(), R_hat, mask=test_mask))
print(relative_loss(R_full.toarray(), R_hat, mask=train_mask))

0.09976071819206714
0.07592997966454759


## n_elem = 0.6

In [177]:
C_ridge = 1e-5
C_group = 2e-4
C_lasso = 10 * C_group
C = (C_lasso, C_group, C_ridge)

eta = 1e1

step_kwargs = {
    "C": C,                 # the regularizr constants (C_lasso, C_group, C_ridge)
    "eta": eta,             # the eta of the ADMM (larger - faster but more unstable)
    "rtol": 1e-4,           # the relative tolerance for stopping the ADMM
    "atol": 1e-7,           # the absolute tolerance
    "method": "cg",         # the method to use in Sub_0
    "n_iterations": 2,      # the number of iterations of the inner ADMM
}

n_iter = 100
K = 10

n_elem = 0.6

In [171]:
interaction_train, interaction_test = sample_from_interactions(interactions, n_elem)

R_train = from_interactions_to_coo(interaction_train, R_shape=R_full.shape).tocsr()
train_mask = R_train.toarray() > 0
test_mask = from_interactions_to_coo(interaction_test, R_shape=R_full.shape).toarray() > 0

/nmnt/media/home/boris/env3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [178]:
problem = IMCProblem(QAObjectiveLoss, X_comb, Y_comb, R_train, n_threads=8)

W_0 = random_state.normal(size=(X_comb.shape[1], K))
H_0 = random_state.normal(size=(Y_comb.shape[1], K))

W, H = W_0.copy(), H_0.copy()

W, H = imc_descent(problem, W, H,
                   step_fn,                  # the inner optimization
                   step_kwargs=step_kwargs,  # asrtguments for the inner optimizer
                   n_iterations=n_iter,      # the number of outer iterations (Gauss-Siedel)
                   n_init_iterations=0,
                   return_history=True,      # Record the evolution of the matrices (W, H)
                   rtol=1e-5,                # relative stopping tolerance for the outer iterations
                   atol=1e-7,                # absolute tolerance
                   verbose=True,             # show the progress bar
                   check_product=True,       # use the product W H' for stopping
                   )

0it [00:00, ?it/s]
100%|██████████| 100/100 [00:07<00:00, 13.28it/s]


In [179]:
R_hat = get_prediction(X_comb, W, H, Y_comb)

print(relative_loss(R_full.toarray(), R_hat, mask=test_mask))
print(relative_loss(R_full.toarray(), R_hat, mask=train_mask))

0.15727194503168554
0.08519146724899301
